In [1]:
from flask import Flask, render_template, request, jsonify, redirect, url_for
from PIL import Image
import numpy as np
from io import BytesIO
import base64
import tensorflow as tf
import os
from sklearn.utils.multiclass import unique_labels
import matplotlib
import cv2
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, precision_recall_curve
from datetime import datetime
import re
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from flask_wtf.recaptcha import RecaptchaField
from wtforms.validators import DataRequired, Email
import requests

2024-10-03 20:06:27.305543: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from flask import Flask, request, jsonify
import re
import random

class ChatBot:
    def __init__(self):
        self.responses = [
            ("Hello!", ["hello", "hi", "hey", "sup", "heyo"], True),
            ("See you!", ["bye", "goodbye"], True),
            ("As of now, there is no known cure for Alzheimer's disease. However, there are treatments and interventions available that can help manage symptoms, slow down the progression of the disease, and improve the quality of life for individuals affected by Alzheimer's.",
             ["how", "do", "I", "cure", "alzhiemer", "what", "is", "the", "cure", "for", "alzhiemer"], True, ["cure", "alzhiemer"]),
            ("I'm doing fine, and you?", ["how", "are", "you", "doing"], True, ["how"]),
            ("Alzhiemer's disease, also known as dementia, is a degenerative neurological condition that progressively impairs cognitive functions, predominantly manifesting as memory loss, difficulties in problem-solving, language impairments, changes in behavior and personality, and ultimately leading to a decline in overall functional abilities and independence.",
             ["what", "is", "do", "you", "know", "alzhiemer"], True, ["what", "alzhiemer"]),
            ("Symptoms of Alzheimer's disease include memory loss, confusion, difficulty with problem-solving, and changes in mood or personality.",
             ["symptoms", "of", "alzhiemer"], True, ["symptoms", "alzhiemer"]),
            ("You are welcome.", ["thank", "thanks", "you"], False, ["thank"]),
            ("Thank you!", ["i", "love", "you"], False, ["love"]),
            ("Sorry, that's all I have", ["give", "me", "more"], False, ["more"]),
            ("Sorry about that.", ["i", "am", "not", "okay", "fine", "doing", "alright"], False, ["i", "not"]),
            ("That's nice to hear!", ["i", "am", "okay", "fine", "doing", "alright"], False, ["i", "am"]),
            ("yeah!", ["oh", "alright", "okay"], True, ["oh", "okay"]),
            ("Sorry, i only have information on Alzheimer's disease", ["give", "advice"], False, ["advice"]),
            ("I can't eat anything because I'm a bot!", ["eat", "eaten"], True, ["you", "eat"]),
            ("yeah!", ["oh", "alright", "okay"], True, ["oh", "okay"]),
            ("To know if you have Alzheimer's, it is important to look out for symptoms like memory loss, confusion, difficulty with problem-solving, and changes in mood or personality. If you have these symptoms, it is advisable to contact a medical personnel...You should make use of our AI model in the upload section to check if you have dementia by just uploading your neural x-ray!",
             ["how", "do", "I", "know", "if", "have", "alzhiemer"], True, ["how", "know", "alzhiemer"]),
            ("Click on the 'get started now' button in the upload section and upload your x-ray to check if you have dementia.",
             ["where", "can", "i", "check", "my", "xray"], True, ["check", "xray"]),
            ("Our AI model has an accuracy of 90%, but it's not perfect. For further assistance, consult a medical professional. Click on the ""Metrics"" link in the navigation bar to get more information on our AI performance metrics.",
             ["how", "accurate", "is", "your", "model", "what", "accuracy", "model", "accuracy"], True, ["accuracy", "model"]),
            ("Mindcare Hospital is a neurological facility which specializes in delivering extensive mental health services such as therapy, counseling and psychiatric care.They provide a nurturing environment for individuals seeking treatment for diverse mental health conditions.",
             ["what", "is", "mindcare", "does"], True, ["what","mindcare"]),
            ("They are located at 709 Covenant University, Ota, Ogun state with their second branch at Ogba, Lagos.",
             ["where", "are", "they", "located"], True, ["where", "located"]),
            ("You could call or send a message to +234-874-248-2362. Click on the Contact link in the navigation bar to get more information on their Contact.", 
             ["what", "is", "their", "contact", "how", "do", "i", "message", "them"], True, ["contact", "message"]),
            ("The time is %s" % datetime.now().strftime("%H:%M"), 
             ["what", "is", "the", "time", "current", "now"], True, ["time"])
        ]
        
        self.mistake_corrections = {
            "thanks": "thank you",
            "Im": "I am",
            "im" : "I am",
            "alziemer" : "alzhiemer's disease",
            "alzheimer" : "alzhiemer's disease",
            "hii" : "hi",
            "hwy" : "hey",
            "alheimer" : "alzhiemer's disease"
        }

    def message_probability(self, user_message, recognised_words, single_response=False, required_words=None):
        message_certainty = 0
        has_required_words = True

        for word in user_message:
            if word in recognised_words:
                message_certainty += 1

        percentage = float(message_certainty) / float(len(recognised_words))

        if required_words:
            for word in required_words:
                if word not in user_message:
                    has_required_words = False
                    break

        if has_required_words or single_response:
            return int(percentage * 100)
        return 0

    def check_all_messages(self, message):
        highest_prob_list = {}

        def response(bot_response, list_of_words, single_response=False, required_words=None):
            probability = self.message_probability(message, list_of_words, single_response, required_words)
            highest_prob_list[bot_response] = probability

        for res in self.responses:
            if len(res) == 3:
                response(res[0], res[1], res[2])
            elif len(res) == 4:
                response(res[0], res[1], res[2], res[3])

        best_match = max(highest_prob_list, key=highest_prob_list.get, default=None)
        return LongResponses.unknown() if highest_prob_list.get(best_match, 0) < 1 else best_match

    def get_response(self, user_message):
        split_message = re.split(r'\s+|[,;?!.-]\s*', user_message.lower())
        corrections = self.correct_mistakes(split_message)
        corrected_message = ' '.join(corrections)
        response = self.check_all_messages(corrections)
        if len(corrections) > 2 and corrections[0] == "what" and corrections[1] == "is" and (corrections[2] == "alzhiemer's" or corrections[2] == "dementia" or corrections[2] == "disease"):
            return "Alzheimer's disease, also known as dementia, is a degenerative neurological condition that progressively impairs cognitive functions, predominantly manifesting as memory loss, difficulties in problem-solving, language impairments, changes in behavior and personality, and ultimately leading to a decline in overall functional abilities and independence."
        
        if corrections != split_message:
            return f"Did you mean: '{corrected_message}'?, please type it again"
        return response
    
    def correct_mistakes(self, message):
        corrected_message = []
        for word in message:
            corrected_message.append(self.mistake_corrections.get(word, word))
        return corrected_message

class LongResponses:
    R_EATING = "I don't like eating anything because I'm a bot obviously!"
    R_ADVICE = "I only have information on Alzheimer's, maybe you should ask a search engine"

    @staticmethod
    def unknown():
        responses = ["I have no answer to this"]
        return random.choice(responses)

In [3]:
app = Flask(__name__)

app.config['SECRET_KEY'] = ''
app.config['RECAPTCHA_PUBLIC_KEY'] = ''
app.config['RECAPTCHA_PRIVATE_KEY'] = ''
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False 

db = SQLAlchemy(app)

class AlzhiemerPatient(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    Name = db.Column(db.String(500), nullable = False)
    Email = db.Column(db.String(100), nullable = False)
    Phone_Number = db.Column(db.String(12), nullable = False)
    Message = db.Column(db.String(1000), nullable = False)
    Date_joined = db.Column(db.Date, default=datetime.utcnow)

    def __repr__(self):
        return f'<AlzhiemerPatient: {self.id}, {self.Name}, {self.Email}, {self.Phone_Number}, {self.Message}, {self.Date_joined}>'

In [4]:
with app.app_context():
    db.create_all()

In [ ]:
bot = ChatBot()

def load_model():
    model = tf.keras.models.load_model('my_finalyear_alzhiemer_project_main_model.keras')
    return model
    
def preprocess_image(image_data):
    image = Image.open(BytesIO(base64.b64decode(image_data)))
    image = image.resize((224, 224)) 
    image = np.array(image) 
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

@app.route('/')
def index():
    return render_template('index.htm', site_key=app.config['RECAPTCHA_PUBLIC_KEY'])

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    response = bot.get_response(user_message)
    return jsonify({'response': response})
    
@app.route('/predict', methods=['POST'])
def predict():
    image_data = request.json['image']
    preprocessed_image = preprocess_image(image_data)
    model = load_model()
    predictions = model.predict(preprocessed_image)
    if predictions > 0.5:
        statistic = predictions * 100
        result = "This x-ray appears to be %.2f percent %s" % (statistic, "NORMAL")
        predicted_label = 1
    else:
        statistic = (1.0 - predictions) * 100
        result = "This x-ray appears to be %.2f percent %s" % (statistic, "DEMENTED")
        predicted_label = 0
    return jsonify({'prediction': result})


@app.route('/submit_form', methods=['POST'])
def submit_form():
    recaptcha_response = request.form['g-recaptcha-response']
    recaptcha_secret = app.config['RECAPTCHA_PRIVATE_KEY']
    recaptcha_verify_url = 'https://www.google.com/recaptcha/api/siteverify'
    recaptcha_data = {
        'secret': recaptcha_secret,
        'response': recaptcha_response
    }
    recaptcha_result = requests.post(recaptcha_verify_url, data=recaptcha_data)
    result = recaptcha_result.json()
    

        
    Names = request.form['name']
    Emails = request.form['email']
    Subjects = request.form['subject']
    Messages = request.form['message']
    with app.app_context():
        existing_record = AlzhiemerPatient.query.filter_by(Email=Emails).first()
        if existing_record:
            return jsonify({'status': 'error', 'message': 'Email already exists'})
        existing_record = AlzhiemerPatient.query.filter_by(Phone_Number=Subjects).first()
        if existing_record:
            return jsonify({'status': 'error', 'message': 'Phone Number already exists'})
        user_message = AlzhiemerPatient(
            Name = Names,
            Email = Emails,
            Phone_Number = Subjects,
            Message = Messages
        )
        db.session.add(user_message)
        db.session.commit()
    return jsonify({'status': 'success', 'message': 'Data sent successfully'})

    


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic3.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/mind_care.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/hekimbaba.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic4.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic5.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/roc_curve.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ic6.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 20:07:16] "GET /static/img/ks_chart.png HTTP/1.1" 200 -
